<a href="https://colab.research.google.com/github/VekhovIO/portfolio/blob/main/LogReg_final_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В этом задании попытаемся спрогнозировать, купит клиент велосипед или нет, обучив логистическую регрессию.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [3]:
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/bike_buyers_clean.csv')

In [4]:
data.head(5)

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


In [5]:
data.shape

(1000, 13)

In [6]:
data.describe(include = 'object')

,Marital Status,Gender,Education,Occupation,Home Owner,Commute Distance,Region,Purchased Bike
count,1000,1000,1000,1000,1000,1000,1000,1000
unique,2,2,5,5,2,5,3,2
top,Married,Male,Bachelors,Professional,Yes,0-1 Miles,North America,No
freq,539,509,306,276,685,366,508,519


Закодируем все категориальные столбцы с двумя категориями следующим образом:  
самая часто встречающаяся категория превращается в 1, другая в 0.

In [7]:
data['Marital Status'] = data['Marital Status'].apply(lambda x: 1 if x == 'Married' else 0)
data['Gender'] = data['Gender'].apply(lambda x: 1 if x == 'Male' else 0)
data['Home Owner'] = data['Home Owner'].apply(lambda x: 1 if x == 'Yes' else 0)
data['Purchased Bike'] = data['Purchased Bike'].apply(lambda x: 1 if x == 'No' else 0)

In [8]:
data = data.drop(columns = ['Education', 'Occupation', 'Commute Distance', 'Region'])

Удалим столбец `ID`, так как он по сути нечисловой.

In [9]:
data = data.drop(columns = 'ID')

Сформируем матрицу объект-признак `X` и вектор `y` с целевой переменной.  
Целевая переменная - это последний столбец, `Purchased Bike`.

In [10]:
X = data.drop(columns = 'Purchased Bike')
y = data['Purchased Bike']

In [11]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = 0.3,  random_state = 42)

Почти всё готово для обучения модели!

Осталось отмасштабировать матрицу `X`, так как линейные модели чувствительны к масштабу данных.

In [12]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

Xtrain_scaled = scaler.fit_transform(Xtrain)
Xtest_scaled = scaler.transform(Xtest)

Xtrain = pd.DataFrame(Xtrain_scaled, columns=Xtrain.columns)
Xtest = pd.DataFrame(Xtest_scaled, columns=Xtest.columns)

Теперь обучим логистическую регрессию на тренировочных данных

In [13]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

model.fit(Xtrain, ytrain)

LogisticRegression()

Сделаем предсказания на тренировочных и на тестовых данных.

In [14]:
ypred = model.predict(Xtest)

Оценим значение accuracy на трейне и на тесте.

In [15]:
from sklearn.metrics import accuracy_score

accuracy_score(ytest, ypred)

0.5766666666666667

Качество модели получилось невысоким, зато модель не переобучена.

Попробуем добавить новых признаков в модель, используя [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Создайте полиномиальные признаки degree = 2.

In [16]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2, include_bias=False)

Xtrain_poly = poly.fit_transform(Xtrain)
Xtest_poly = poly.transform(Xtest)

Xtrain = pd.DataFrame(Xtrain_poly, columns=poly.get_feature_names_out(Xtrain.columns))
Xtest = pd.DataFrame(Xtest_poly, columns=poly.get_feature_names_out(Xtest.columns))

Заново обучим логистическую регрессию, уже на расширенной матрице признаков

In [17]:
model.fit(Xtrain, ytrain)

ypred = model.predict(Xtest)

accuracy_score(ytest, ypred)

0.62

Появились новые требования от заказчика!

Заказчик просит, чтобы:
*  доля найденных моделью потенциальных покупателей была максимальной
*  accuracy при этом была не ниже, чем 0.6 (отклонения *accuracy* на тестовых данных на $\pm 0.05$ допустимы).

Сначала посмотрим, какие значения *recall* и *accuracy* имеют предсказния модели на тесте с классами, предсказанными по умолчанию (методом `predict`).

In [18]:
from sklearn.metrics import recall_score

recall_score(ytest, ypred)

0.6891891891891891

Подберём порог для перевода вероятностей в классы, чтобы оптимизировать требуемые метрики!

Разобъем тренировочные данные на трейн и валидацию, чтобы по валидационной части подбирать порог.

In [19]:
XtrainS, Xval, ytrainS, yval = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)

XtrainS.shape, Xval.shape

((490, 35), (210, 35))

In [20]:
from sklearn.metrics import recall_score

RecMax = -1
BestThr = -1
BestAcc = -1

for thr in np.arange(0, 1, 0.01):
    y_val_pred = (yval >= thr).astype(int)

    recall = recall_score(yval, y_val_pred)
    accuracy = accuracy_score(yval, y_val_pred)

    if accuracy > 0.6 and recall > RecMax:
        RecMax = recall
        BestThr = thr
        BestAcc = accuracy


print (BestThr, RecMax, BestAcc)

0.01 1.0 1.0


Теперь заново обучим модель на исходных тренировочных данных (`Xtrain`, `ytrain`), предскажим вероятности на тесте и переведем их в классы по найденному порогу.

In [21]:
model.fit(Xtrain, ytrain)

y_test_probs = model.predict_proba(Xtest)[:, 1]

y_test_pred = (y_test_probs >= BestThr).astype(int)

recall_test = recall_score(ytest, y_test_pred)

recall_test_rounded = round(recall_test, 1)

print(f"Recall на тестовых данных: {recall_test_rounded}")

Recall на тестовых данных: 1.0


При помощи подбора порога удалось сильно увеличить значение *recall*!  
Однако, как видно, на тесте не удалось сохранить условие $accuracy \geq 0.6$ (но в допустимые рамки уложились!)

Это свидетельство небольшого переобучения модели. Однако в сухом остатке имеет значительное увеличение полноты, что является приоритетом для заказчика.